In [88]:
import os
import re
import json
import pandas as pd
from src.utils import read_config
from prompts import gemini_prompts
import google.generativeai as genai
import google.ai.generativelanguage as glm

from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS, DistanceStrategy
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

from neo4j import GraphDatabase

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
organized_result = pd.read_csv('temp_result/organized_result_2.csv')
organized_result.head()

,source,target,reason,fact,keywords
0,34829.pdf,行政院原住民族委員會及花蓮縣政府,花蓮區漁會辦理「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機採購案」，未能確實...,據立法委員函轉民眾陳訴行政院原住民族委員會(下稱原民會)補助花蓮區漁會給二予漁民架設衛星導航...,"['政府', '行政院', '案件', '原住民', '計畫', '漁探機', '本院', ..."
1,34838.pdf,行政院衛生署,為行政院衛生署辦理九十至九十三年度補、捐助計畫，經該署檢討至少計一二五項計畫應以委辦而竟以補...,本案係立法院鄭委員三元向本院陳訴略以「行政院衛生署將應以委辦執行之經費，改由補助費方式辦理，...,"['計畫', '捐助', '年度', '行政院', '案件', '意旨', '下列', '方..."
2,36946.pdf,台北縣萬里鄉公所、台北市政府、基隆市政府、台北縣政府,台北市政府、基隆市政府及台北縣政府未善盡土地使用管制、環境污染稽查及申報資料勾稽、查核之責，...,本案緣陳訴人檢附採證照片陳訴台北市政府核准設立之新生代環境清潔公司（下二稱新生代公司）、統徠...,"['政府', '公司', '清除', '垃圾', '顯有', '核准', '必要性', '欠..."
3,36997.pdf,行政院國軍退除役官兵輔導委員會台北榮民總醫院,行政院國軍退除役官兵輔導委員會台北榮民總醫院辦理「醫學科技大樓法定停車場工程」興建發包作業，...,本案前經審計部函報行政院國軍退除役官兵輔導委員會（以下簡稱退輔會）台北榮民總醫院（以下簡稱台...,"['工程', '提升', '於九', '法定', '完成', '台北', '法令', '方式..."
4,37008.pdf,行政院國軍退除役官兵輔導委員會,為行政院國軍退除役官兵輔導委員會為改善所屬十一所榮民醫院及十八所安養機構之污水處理廠及衛生下...,行政院國軍退除役官兵輔導委員會﹙以下簡稱退輔會﹚為改善所屬十一所榮民醫院及十八所安養機構之廢...,"['工程', '政府', '下水道', '方式', '核有', '污水', '蒐集', '曲..."


In [3]:
configs = read_config('.env/configs.json')
api_key =configs['g_key']
os.environ["GOOGLE_API_KEY"] = api_key
genai.configure(api_key=api_key)

In [4]:
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    convert_system_message_to_human=True,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)

In [5]:
kg_reseult = organized_result.copy()
for i, row in kg_reseult.iterrows():
    
    triple_prompt = PromptTemplate.from_template(gemini_prompts.TRIPLE_PROMPT)
    chain = triple_prompt | llm
    triple_data = {
        "reference": row['reason']
    }

    triple_response = chain.invoke(triple_data)

    triple_response_result = re.sub(r'[```json\n]', '', triple_response.content).replace("實體", "entity").replace("關係", "relationship").split("、")[0]

    to_json_prompt = PromptTemplate.from_template(gemini_prompts.TO_JSON_PROMPT)
    chain = to_json_prompt | llm
    data = {
        "reference": triple_response_result
    }

    to_json_response = chain.invoke(data)
    
    
    triple_json_result = re.sub(r'[\n]','',to_json_response.content)

    # triple_json_result = json.loads(f'[{triple_json_result}]')
    
    kg_reseult.loc[kg_reseult.index==i, 'kg'] = str(triple_json_result)
    
    

c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Co

In [74]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 500,
    length_function= len
)

In [102]:
kg_reseult = kg_reseult.rename(columns={'ids':'source'})

In [103]:
reason_loader = DataFrameLoader(kg_reseult, page_content_column='reason')
reason_data = reason_loader.load()

# target_loader = DataFrameLoader(organized_result, page_content_column='target')
# target_data = target_loader.load()

reason_documents = text_splitter.transform_documents(reason_data)
print("已為所有文件進行 chunk ",len(reason_documents),"筆")

已為所有文件進行 chunk  75 筆


In [76]:
reason_documents[0].metadata

{'ids': '34829.pdf',
 'target': '行政院原住民族委員會及花蓮縣政府',
 'fact': '據立法委員函轉民眾陳訴行政院原住民族委員會(下稱原民會)補助花蓮區漁會給二予漁民架設衛星導航系統疑有弊端，惟該會遲未盡查處之責，涉有違失等情經本院調查結果，行政院原住民族委員會及花蓮縣政府辦理補助花蓮區漁會採購「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機」乙案，確有下列違失一、花蓮區漁會辦理「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機採購案」，未能確實依政府採購法之規定辦理，採購作業錯誤叢生，補助機關行政院原住民族委員會及花蓮縣政府原住民行政局顯然疏於監督，核有未當，應予檢討改進(一)查行政院原住民族委員會民國(下同)九十二年度補助花蓮縣政府支付花蓮區漁會辦理部落產業發展計畫｜東海岸原住民漁業輔導計畫經費新台幣(下同)一七七萬元，並納入該府預算執行，其中花蓮區漁會辦理「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機採購案」，於九十二年十月十四日開標，由友漁企業公司以一二五萬元得標，惟經查該漁會辦理該採購案違反政府採購法諸多規定，核有下列缺失１、該漁會收受三家廠商投標文件，未於標封袋登記廠商送達方式與時間，亦無簽收紀錄可稽；審查廠商投標作業，僅於證件封袋上予以勾選，無審查之人員簽名負責；又未能提供廠商繳納押標金之紀錄憑核，對於其繳交之額度、支票號碼及領回日期等資料，均無從查核２、該採購案並非特殊或鉅額採購，卻限定投標資格為資本額新台幣五百萬元以上之廠商，上開廠商須具有相當財力之規定，核與政府採購法第三十六條第二項暨「投三標廠商資格與特殊或巨額採購認定標準」第五條第一項第三款規定不符核有非屬特殊或鉅額採購，卻限定財力之特定資格，不當限制廠商競爭之情事，顯已違反採購法第三十七條「機關訂定投標廠商之資格，不得不當限制競爭，並以確認廠商具備履行契約所必須之能力者為限」之規定３、該採購案招標公告刊登標的名稱為「漁船衛星定位儀及通訊系統工程採購」及標的分類為「漁業工程」，惟查採購合約名稱為「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機」，且實際辦理內容則為設備類之財物採購，核有採購屬性刊登錯誤４、該採購案未得標廠商陸青科技股份有限公司，其營利事業登記證、經濟部公司執照及交通部電信管制器材經營許可執照，

In [14]:
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

In [10]:
# vector_db_path = "./vector_db_2/"

# if not os.path.exists(vector_db_path):
#     os.makedirs(vector_db_path)
#     print(f"Created folder: {vector_db_path}")
    
# vector_store = FAISS.from_documents(reason_documents, embeddings, normalize_L2=True, distance_strategy=DistanceStrategy.MAX_INNER_PRODUCT)
# vector_store.save_local(folder_path=vector_db_path)

In [104]:
id_list = kg_reseult['source'].to_list()

In [106]:
vector_db_path = "./chroma_db/"
chroma_collection_name = "corretion_v1"
if not os.path.exists(vector_db_path):
    os.makedirs(vector_db_path)
    print(f"Created folder: {vector_db_path}")
    
vector_store = Chroma.from_documents(documents=reason_documents, 
                                     persist_directory=vector_db_path, 
                                     collection_name=chroma_collection_name, 
                                     ids=id_list,
                                     embedding=embeddings)

In [110]:
vectorstore = Chroma(persist_directory=vector_db_path, collection_name=chroma_collection_name, embedding_function=embeddings)
vectorstore._collection.count()

75

In [105]:
# vector_store.delete_collection()

In [7]:
kg_reseult.to_csv('temp_result/result_with_kg.csv', index=False)

In [83]:
def transform_kg_to_json(df, column_name):
    def validate_and_transform(kg_value):
        try:
            triple_json_result = re.sub(r'[\n]', '', kg_value)
            triple_json_result = json.loads(f'[{triple_json_result}]')
            return triple_json_result
        except:
            return None

    df[f'{column_name}_json'] = df[column_name].apply(validate_and_transform)
    return df

In [168]:
df_test = kg_reseult[kg_reseult['kg_json'].notnull()]
df_test = transform_kg_to_json(df_test, 'kg')

C:\Users\sean.chang\AppData\Local\Temp\ipykernel_4404\2243806963.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{column_name}_json'] = df[column_name].apply(validate_and_transform)


In [169]:
URI = "bolt://localhost:7688"
AUTH = ("neo4j", "!QAZ2wsx")

with GraphDatabase.driver(URI, auth=AUTH, encrypted=False) as driver:
    driver.verify_connectivity()

In [214]:
def create_relationships(tx, data):
    # print(data)
    for item in data:
        try:
            tx.run("MERGE (a:Entity {name: $entity1}) "
                "MERGE (b:Entity {name: $entity2}) "
                "MERGE (a)-[r:RELATIONSHIP {type: $relationship}]->(b)",
                entity1=item[0]["entity1"], relationship=item[0]["relationship"], entity2=item[0]["entity2"])
        except:
            pass

In [212]:
row = df_test[df_test.index == 2]
data = row.kg_json
data

2    [{'entity1': '台北市政府', 'relationship': '未善盡', '...
Name: kg_json, dtype: object

In [213]:
for item in data:
    try:
        print(item)
        print(item[0]["entity1"])
        
    except:
        pass

[{'entity1': '台北市政府', 'relationship': '未善盡', 'entity2': '土地使用管制'}]
台北市政府


In [215]:
for i in df_test.index:
    row = df_test[df_test.index == i]
    data = row.kg_json
    with GraphDatabase.driver(uri=URI, auth=("neo4j", "neo4j")) as driver:
        with driver.session() as session:
            session.write_transaction(create_relationships, data)

In [216]:
def fetch_data(tx):
    result = tx.run("MATCH (n:Entity)-[r:RELATIONSHIP]->(m:Entity) "
                    "RETURN n.name, r.type, m.name")
    for record in result:
        print(f"{record['n.name']} {record['r.type']} {record['m.name']}")

with GraphDatabase.driver(uri=URI, auth=("neo4j", "neo4j")) as driver:  # Replace auth credentials as necessary
    with driver.session() as session:
        session.read_transaction(fetch_data)


南投縣草屯鎮公所 執行 過渡時期緊急垃圾處理計畫委託焚化處理案
嘉義市政府 辦理 嘉義市華興橋改建工程周邊道路(許厝庄12米計畫道路工程)
交通部臺灣鐵路管理局（臺鐵局） 辦理 「工程建設暨政策網路行銷採購」
衛生福利部澎湖醫院 未依政府採購法及契約規定 辦理107年停車場設備租賃等採購案
文化部所屬文化資產局 辦理 水下文化資產調查研究工作相關儀器設備案
雲林縣褒忠鄉公所 辦理 111年度「褒忠鄉大手牽小手為愛一起走」活動
空軍司令部 辦理 採購案
新竹縣新豐鄉公所 未審慎辦理 清潔隊人員出缺之甄補作業
台電公司 審議 立式車床採購規格
文化部 洽請 臺北市政府
花蓮區漁會 辦理 三機一體彩色漁探衛星導航電子海圖儀及船用DSB無線電對講機採購案
台北市政府 未善盡 土地使用管制
行政院國軍退除役官兵輔導委員會台北榮民總醫院 辦理 「醫學科技大樓法定停車場工程」興建發包作業
行政院國軍退除役官兵輔導委員會 未經蒐集商情資訊 十一所榮民醫院及十八所安養機構之污水處理廠及衛生下水道工程
教育部 辦理 二○○一年國際生物奧林匹亞競賽參賽計畫
卓蘭鎮公所 辦理 卓蘭鎮老庄區域排水震災修復工程
交通部台灣鐵路管理局 未能善盡職責詳實審查 廠商投標文件
台南市政府 怠未依計畫補助原則和程序 環保庫錢爐
國防部中山科學研究院 編列違反規定 國防部
中華郵政股份有限公司 違反 勞動基準法相關規定
科技部 排除 ○○公司
國立臺灣師範大學 辦理 林口校區資訊與教學大樓新建工程案
原能會 委託 清華大學
東華大學 未詳予審查 繼受廠商之履約資格
國防部 所屬 海軍司令部
國防部陸軍司令部 辦理 「步兵多武器射擊模擬系統」採購案
國防部 辦理 化學消防車採購案
新竹縣政府 接受補助 內政部營建署
南投縣政府 辦理 河川土石疏濬作業
台灣電力股份有限公司高雄興達電廠 辦理 「興#3
國軍退除役官兵輔導委員會花蓮縣榮民服務處 辦理 「100-101年度花蓮地區單身亡故榮民殯葬事務採購案」
台灣中油股份有限公司 採購 液化天然氣LNG
新竹縣五峰鄉公所 辦理 勞務及工程採購案
高雄縣鳳山市文山國民小學 違反 九年一貫課程綱要
財團法人國家實驗研究院國家太空中心主任吳作樂等 辦理 台灣自主衛星ARGO計畫
前行政院青年輔導委員會 辦理 「一百年青年政策大聯盟公共參與網站暨網路經營計畫」
行政院農

In [198]:
def clear_neo4j_database(uri, auth):
    driver = GraphDatabase.driver(uri, auth=auth)
    
    def clear_db(tx):
        tx.run("MATCH (n) DETACH DELETE n")

    with driver.session() as session:
        session.write_transaction(clear_db)
    
    driver.close()
clear_neo4j_database(URI, AUTH)